# Импорт библиотек

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import requests
from datetime import datetime
import time

# Составление датасета

In [26]:
POLLUTANTS = ['pm25', 'pm10', 'so2', 'no2', 'co', 'no']
CITIES = ['Amsterdam', 'Paris', 'Frankfurt', 'Munich', 'Prague', 'Warsaw']
COUNTRIES = ['FR', 'PL', 'DE', 'CZ']
DATE_FROM = '2021-01-01'
DATE_TO = '2024-01-01'
url_measurments = "https://api.openaq.org/v1/measurements"
url_cities = "https://api.openaq.org/v1/cities"
url_locations = "https://api.openaq.org/v1/locations" 
date_range = pd.date_range(start=DATE_FROM, end=DATE_TO, freq='ME')

with open('apikey', "r") as keyfile:
    HEADERS = {}
    HEADERS['x-api-key'] = keyfile.readline().strip()

In [12]:
all_results = []
 
for pollutant in POLLUTANTS:
    for city in CITIES:
        for month_end in date_range:
            start_date = month_end.replace(day=1)
            end_date = month_end

            print(f"Начало: {start_date.strftime('%Y-%m-%d')}, Конец: {end_date.strftime('%Y-%m-%d')}")

            params = {
                'city': city,
                'parameter': pollutant,
                'date_from': start_date.strftime('%Y-%m-%dT00:00:00Z'),
                'date_to': end_date.strftime('%Y-%m-%dT23:59:59Z'),
                'page' : 1,
                'limit': 100
            }

            while True:
                response = requests.get(url, headers=HEADERS, params=params)
                
                if response.status_code == 429:
                    print("Превышено количество запросов. Ожидание 5 секунд...")
                    time.sleep(5)
                    continue
                
                if response.status_code != 200:
                    print(f"Ошибка: {response.status_code}, {response.text}")
                    break

                data = response.json()
                results = data.get('results', [])
                
                if not results:
                    print(f"Нет данных для {city}, {pollutant} в период с {start_date} по {end_date}")
                    break

                all_results.extend(results)

                if 'meta' in data and 'next' in data['meta']:
                    params['page'] = data['meta']['next']
                else:
                    break
                
                time.sleep(2)

# Преобразуем список результатов в DataFrame
df = pd.DataFrame(all_results)

# Сохранение в CSV для дальнейшего использования
df.to_csv('Amsterdam-Vondelpark.csv', index=False)
print("Данные сохранены в 'openaq_measurements.csv'")

Начало: 2021-01-01, Конец: 2021-01-31
Нет данных для Amsterdam-Vondelpark, pm25 в период с 2021-01-01 00:00:00 по 2021-01-31 00:00:00
Начало: 2021-02-01, Конец: 2021-02-28
Нет данных для Amsterdam-Vondelpark, pm25 в период с 2021-02-01 00:00:00 по 2021-02-28 00:00:00
Начало: 2021-03-01, Конец: 2021-03-31
Нет данных для Amsterdam-Vondelpark, pm25 в период с 2021-03-01 00:00:00 по 2021-03-31 00:00:00
Начало: 2021-04-01, Конец: 2021-04-30
Нет данных для Amsterdam-Vondelpark, pm25 в период с 2021-04-01 00:00:00 по 2021-04-30 00:00:00
Начало: 2021-05-01, Конец: 2021-05-31
Нет данных для Amsterdam-Vondelpark, pm25 в период с 2021-05-01 00:00:00 по 2021-05-31 00:00:00
Начало: 2021-06-01, Конец: 2021-06-30
Нет данных для Amsterdam-Vondelpark, pm25 в период с 2021-06-01 00:00:00 по 2021-06-30 00:00:00
Начало: 2021-07-01, Конец: 2021-07-31
Нет данных для Amsterdam-Vondelpark, pm25 в период с 2021-07-01 00:00:00 по 2021-07-31 00:00:00
Начало: 2021-08-01, Конец: 2021-08-31
Нет данных для Amsterdam

In [35]:
all_cities = []

for i in range(0, len(COUNTRIES), 2):
    batch_countries = COUNTRIES[i:i + 2]
    
    params = {
        'country': batch_countries,
        'limit': 10,
        'page': 1
    }
    
    print(f"Запрос городов для стран: {batch_countries}")
    

    response = requests.get(url_cities, headers=HEADERS, params=params)
    
    if response.status_code == 429:
        print("Превышено количество запросов. Ожидание 5 секунд...")
        time.sleep(5)
        continue

    if response.status_code != 200:
        print(f"Ошибка: {response.status_code}, {response.text}")
        break
    
    data = response.json()
    results = data.get('results', [])
    print(results)
        
        

Запрос городов для стран: France
Ошибка: 422, "1 validation error for CitiesV1\ncountry\n  List should have at least 2 items after validation, not 1 [type=too_short, input_value=['France'], input_type=list]\n    For further information visit https://errors.pydantic.dev/2.1.2/v/too_short"


In [22]:
all_locations = []

for city in CITIES:
    params = {
        'city': city,
        'limit': 10,
    }
    
    print(f"Запрос локаций для города: {city}")
    
    while True:
        response = requests.get(url_locations, headers=HEADERS, params=params)
        
        if response.status_code == 429:
            print("Превышено количество запросов. Ожидание 5 секунд...")
            time.sleep(5)
            continue

        if response.status_code != 200:
            print(f"Ошибка: {response.status_code}, {response.text}")
            break
        
        data = response.json()
        results = data.get('results', [])
        
        if not results:
            print(f"Нет данных для {city}")
            break
        
        all_locations.extend([loc['location'] for loc in results])

        if 'meta' in data and 'next' in data['meta']:
            params['page'] = data['meta']['next']
        else:
            break

        time.sleep(2) 

print(all_locations)

Запрос локаций для города: Amsterdam
Запрос локаций для города: Paris
Запрос локаций для города: Frankfurt
Нет данных для Frankfurt
Запрос локаций для города: Munich
Нет данных для Munich
Запрос локаций для города: Prague
Нет данных для Prague
Запрос локаций для города: Warsaw
Нет данных для Warsaw
['Amsterdam-Haarlemmerweg', 'Amsterdam-Vondelpark', 'Amsterdam-Westerpark', 'Amsterdam-Kantershof (Zuid Oost)', 'Amsterdam-Sportpark Ookmeer (Osdorp)', 'Amsterdam-Nieuwendammerdijk', 'Amsterdam-Einsteinweg', 'Amsterdam-Hoogtij', 'Amsterdam-Jan van Galenstraat', 'Amsterdam-Stadhouderskade', 'FR04012']


In [8]:
df

,location,parameter,value,date,unit,coordinates,country,city
0,Amsterdam-Vondelpark,pm25,11.4,"{'utc': '2021-02-27T03:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 52.359714000013874, 'longitude': ...",NL,None
1,Amsterdam-Vondelpark,pm25,13.2,"{'utc': '2021-04-30T04:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 52.359714000013874, 'longitude': ...",NL,None
2,Amsterdam-Vondelpark,pm25,7.6,"{'utc': '2021-05-31T14:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 52.359714000013874, 'longitude': ...",NL,None
3,Amsterdam-Vondelpark,pm25,11.0,"{'utc': '2021-06-30T21:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 52.359714000013874, 'longitude': ...",NL,None
4,Amsterdam-Vondelpark,pm25,50.9,"{'utc': '2021-07-25T03:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 52.359714000013874, 'longitude': ...",NL,None
...,...,...,...,...,...,...,...,...
161,FR04012,no,29.1,"{'utc': '2023-08-31T23:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 48.8277822163547, 'longitude': 2....",FR,None
162,FR04012,no,62.1,"{'utc': '2023-09-30T23:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 48.8277822163547, 'longitude': 2....",FR,None
163,FR04012,no,30.9,"{'utc': '2023-10-31T23:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 48.8277822163547, 'longitude': 2....",FR,None
164,FR04012,no,29.5,"{'utc': '2023-11-30T23:00:00+00:00', 'local': ...",µg/m³,"{'latitude': 48.8277822163547, 'longitude': 2....",FR,None


In [6]:
df.date.unique()

TypeError: unhashable type: 'dict'